In [2]:
import numpy as np
import pandas as pd
import pymongo
from bson.objectid import ObjectId
from math import isnan
DATABASE_URL = "mongodb://localhost:27018"
DATABASE_NAME = "mojeWideloDb"
USER_COLLECTION_NAME = "users"
VIDEO_COLLECTION_NAME = "videoMetadata"
REACTION_COLLECTION_NAME = "reactions"
SUBSCRIPTION_COLLECTION_NAME = "subscriptions"
HISTORY_COLLECTION_NAME = "history"


In [66]:
db_client = pymongo.MongoClient(DATABASE_URL)
db = db_client[DATABASE_NAME]
video_collection = pd.DataFrame(list(db[VIDEO_COLLECTION_NAME].find()))
user_collection = pd.DataFrame(list(db[USER_COLLECTION_NAME].find()))
reaction_collection = pd.DataFrame(list(db[REACTION_COLLECTION_NAME].find()))
subscription_collection = pd.DataFrame(list(db[SUBSCRIPTION_COLLECTION_NAME].find()))
history_collection = pd.DataFrame(list(db[HISTORY_COLLECTION_NAME].find()))

In [27]:
# merge users with histories
users_history = history_collection.merge(user_collection, on="_id")

# process one user history
user_id = ObjectId("6425dbadc7611003a102a7ab")
user_history_index = history_collection["WatchedVideos"][history_collection["_id"] == user_id].index[0]
# todo: do not use subscriptions? we already implemented subs page... (maybe hide subscribed channels?)
user_subscriptions = subscription_collection[subscription_collection["SubscriberId"] == user_id]
user_reactions = reaction_collection[reaction_collection["UserId"] == user_id]
watched_video = pd.DataFrame(history_collection["WatchedVideos"][user_history_index].copy())
user_watched_videos = watched_video.merge(video_collection, left_on="VideoId", right_on="_id").merge(user_reactions, left_on="VideoId", right_on="VideoId", how="left").merge(user_subscriptions, left_on="AuthorId", right_on="CreatorId", how="left").loc[:,["Date", "Tags_x", "ReactionType","SubscriberId"]]
user_watched_videos["Subscribes"] = user_watched_videos.loc[:,"SubscriberId"].transform(lambda x: not pd.isnull(x))
user_watched_videos = user_watched_videos.drop(columns=["SubscriberId"])
user_watched_videos

,Date,Tags_x,ReactionType,Subscribes
0,2023-05-23 16:25:37.391000,NaN,1,True
1,2023-05-23 16:25:38.497000,NaN,1,True
2,2023-05-23 16:25:40.133000,NaN,1,True
3,0001-01-01 00:00:00,[],1,True
4,0001-01-01 00:00:00,[],1,True
5,0001-01-01 00:00:00,[],1,True
6,2023-05-23 16:56:57.438000,[],1,True
7,2023-05-23 16:57:05.996000,[],1,True
8,2023-05-23 16:57:07.285000,[],1,True
9,2023-05-23 16:57:49.469000,"[rozrywka, podróżowanie, technologia, zdrowie]",1,True


In [31]:
watched_video.merge(video_collection, left_on="VideoId", right_on="_id").merge(user_reactions, left_on="VideoId", right_on="VideoId", how="left").merge(user_subscriptions, left_on="AuthorId", right_on="CreatorId", how="left")

,VideoId,Date,Tags_x,_id_x,Title,Description,Thumbnail,Tags_y,Visibility,AuthorId,...,ProcessingProgress,UploadDate,EditDate,Duration,_id_y,UserId,ReactionType,_id,CreatorId,SubscriberId
0,646ce170ea399a31bc7dc3be,2023-05-23 16:25:37.391000,NaN,646ce170ea399a31bc7dc3be,dsaads,,None,"[rozrywka, podróżowanie, technologia, zdrowie]",0,6425dbadc7611003a102a7ab,...,6,2023-05-23 15:53:20.781,2023-05-23 16:57:49.272,00:10,6431b96a0002ae73942404d8,6425dbadc7611003a102a7ab,1,646415141406eb2fb5320910,6425dbadc7611003a102a7ab,6425dbadc7611003a102a7ab
1,646ce170ea399a31bc7dc3be,2023-05-23 16:25:38.497000,NaN,646ce170ea399a31bc7dc3be,dsaads,,None,"[rozrywka, podróżowanie, technologia, zdrowie]",0,6425dbadc7611003a102a7ab,...,6,2023-05-23 15:53:20.781,2023-05-23 16:57:49.272,00:10,6431b96a0002ae73942404d8,6425dbadc7611003a102a7ab,1,646415141406eb2fb5320910,6425dbadc7611003a102a7ab,6425dbadc7611003a102a7ab
2,646ce170ea399a31bc7dc3be,2023-05-23 16:25:40.133000,NaN,646ce170ea399a31bc7dc3be,dsaads,,None,"[rozrywka, podróżowanie, technologia, zdrowie]",0,6425dbadc7611003a102a7ab,...,6,2023-05-23 15:53:20.781,2023-05-23 16:57:49.272,00:10,6431b96a0002ae73942404d8,6425dbadc7611003a102a7ab,1,646415141406eb2fb5320910,6425dbadc7611003a102a7ab,6425dbadc7611003a102a7ab
3,646ce170ea399a31bc7dc3be,0001-01-01 00:00:00,[],646ce170ea399a31bc7dc3be,dsaads,,None,"[rozrywka, podróżowanie, technologia, zdrowie]",0,6425dbadc7611003a102a7ab,...,6,2023-05-23 15:53:20.781,2023-05-23 16:57:49.272,00:10,6431b96a0002ae73942404d8,6425dbadc7611003a102a7ab,1,646415141406eb2fb5320910,6425dbadc7611003a102a7ab,6425dbadc7611003a102a7ab
4,646ce170ea399a31bc7dc3be,0001-01-01 00:00:00,[],646ce170ea399a31bc7dc3be,dsaads,,None,"[rozrywka, podróżowanie, technologia, zdrowie]",0,6425dbadc7611003a102a7ab,...,6,2023-05-23 15:53:20.781,2023-05-23 16:57:49.272,00:10,6431b96a0002ae73942404d8,6425dbadc7611003a102a7ab,1,646415141406eb2fb5320910,6425dbadc7611003a102a7ab,6425dbadc7611003a102a7ab
5,646ce170ea399a31bc7dc3be,0001-01-01 00:00:00,[],646ce170ea399a31bc7dc3be,dsaads,,None,"[rozrywka, podróżowanie, technologia, zdrowie]",0,6425dbadc7611003a102a7ab,...,6,2023-05-23 15:53:20.781,2023-05-23 16:57:49.272,00:10,6431b96a0002ae73942404d8,6425dbadc7611003a102a7ab,1,646415141406eb2fb5320910,6425dbadc7611003a102a7ab,6425dbadc7611003a102a7ab
6,646ce170ea399a31bc7dc3be,2023-05-23 16:56:57.438000,[],646ce170ea399a31bc7dc3be,dsaads,,None,"[rozrywka, podróżowanie, technologia, zdrowie]",0,6425dbadc7611003a102a7ab,...,6,2023-05-23 15:53:20.781,2023-05-23 16:57:49.272,00:10,6431b96a0002ae73942404d8,6425dbadc7611003a102a7ab,1,646415141406eb2fb5320910,6425dbadc7611003a102a7ab,6425dbadc7611003a102a7ab
7,646ce170ea399a31bc7dc3be,2023-05-23 16:57:05.996000,[],646ce170ea399a31bc7dc3be,dsaads,,None,"[rozrywka, podróżowanie, technologia, zdrowie]",0,6425dbadc7611003a102a7ab,...,6,2023-05-23 15:53:20.781,2023-05-23 16:57:49.272,00:10,6431b96a0002ae73942404d8,6425dbadc7611003a102a7ab,1,646415141406eb2fb5320910,6425dbadc7611003a102a7ab,6425dbadc7611003a102a7ab
8,646ce170ea399a31bc7dc3be,2023-05-23 16:57:07.285000,[],646ce170ea399a31bc7dc3be,dsaads,,None,"[rozrywka, podróżowanie, technologia, zdrowie]",0,6425dbadc7611003a102a7ab,...,6,2023-05-23 15:53:20.781,2023-05-23 16:57:49.272,00:10,6431b96a0002ae73942404d8,6425dbadc7611003a102a7ab,1,646415141406eb2fb5320910,6425dbadc7611003a102a7ab,6425dbadc7611003a102a7ab
9,646ce170ea399a31bc7dc3be,2023-05-23 16:57:49.469000,"[rozrywka, podróżowanie, technologia, zdrowie]",646ce170ea399a31bc7dc3be,dsaads,,None,"[rozrywka, podróżowanie, technologia, zdrowie]",0,6425dbadc7611003a102a7ab,...,6,2023-05-23 15:53:20.781,2023-05-23 16:57:49.272,00:10,6431b96a0002ae73942404d8,6425dbadc7611003a102a7ab,1,646415141406eb2fb5320910,6425dbadc7611003a102a7ab,6425dbadc7611003a102a7ab


In [86]:
video_count = len(video_collection["AuthorId"])
movie_matrix = np.zeros((video_count, video_count), float)
for i in range(video_count):
    for j in range(video_count):
        tags1, tags2 = video_collection["Tags"][i], video_collection["Tags"][j]
        min_length = (min(len(tags1), len(tags2)))
        if(i == j or min_length == 0):
            print(min_length)
            continue
        movie_matrix[i,j] = len(np.intersect1d(tags1, tags2)) / min_length
movie_matrix

0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
2
0
0
4


array([[0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0.]])

In [81]:
(min(len(video_collection["Tags"][2]), len(video_collection["Tags"][3]))) == 0


False